This exercise will require you to pull some data from the Qunadl API. Qaundl is currently the most widely used aggregator of financial market data.

As a first step, you will need to register a free account on the http://www.quandl.com website.

After you register, you will be provided with a unique API key, that you should store:

In [1]:
# Store the API key as a string - according to PEP8, constants are always named in all upper case
API_KEY = ''

Qaundl has a large number of data sources, but, unfortunately, most of them require a Premium subscription. Still, there are also a good number of free datasets.

For this mini project, we will focus on equities data from the Frankfurt Stock Exhange (FSE), which is available for free. We'll try and analyze the stock prices of a company called Carl Zeiss Meditec, which manufactures tools for eye examinations, as well as medical lasers for laser eye surgery: https://www.zeiss.com/meditec/int/home.html. The company is listed under the stock ticker AFX_X.

You can find the detailed Quandl API instructions here: https://docs.quandl.com/docs/time-series

While there is a dedicated Python package for connecting to the Quandl API, we would prefer that you use the *requests* package, which can be easily downloaded using *pip* or *conda*. You can find the documentation for the package here: http://docs.python-requests.org/en/master/ 

Finally, apart from the *requests* package, you are encouraged to not use any third party Python packages, such as *pandas*, and instead focus on what's available in the Python Standard Library (the *collections* module might come in handy: https://pymotw.com/3/collections/ ).
Also, since you won't have access to DataFrames, you are encouraged to us Python's native data structures - preferably dictionaries, though some questions can also be answered using lists.
You can read more on these data structures here: https://docs.python.org/3/tutorial/datastructures.html

Keep in mind that the JSON responses you will be getting from the API map almost one-to-one to Python's dictionaries. Unfortunately, they can be very nested, so make sure you read up on indexing dictionaries in the documentation provided above.

In [2]:
# First, import the relevant modules
import requests
import json
import statistics

In [3]:
# Now, call the Quandl API and pull out a small sample of the data (only one day) to get a glimpse
# into the JSON structure that will be returned
r_sample = requests.get('https://www.quandl.com/api/v3/datasets/FSE/AFX_X/data.json?start_date=2019-06-07&end_date=2019-06-07&api_key=' + API_KEY)
j_sample = r_sample.json()


In [4]:
# Inspect the JSON structure of the object you created, and take note of how nested it is,
# as well as the overall structure
j_sample

{'dataset_data': {'limit': None,
  'transform': None,
  'column_index': None,
  'column_names': ['Date',
   'Open',
   'High',
   'Low',
   'Close',
   'Change',
   'Traded Volume',
   'Turnover',
   'Last Price of the Day',
   'Daily Traded Units',
   'Daily Turnover'],
  'start_date': '2019-06-07',
  'end_date': '2019-06-07',
  'frequency': 'daily',
  'data': [['2019-06-07',
    None,
    87.2,
    85.45,
    86.95,
    None,
    138663.0,
    12010778.0,
    None,
    None,
    None]],
  'collapse': None,
  'order': None}}

These are your tasks for this mini project:

1. Collect data from the Franfurt Stock Exchange, for the ticker AFX_X, for the whole year 2017 (keep in mind that the date format is YYYY-MM-DD).
2. Convert the returned JSON object into a Python dictionary.
3. Calculate what the highest and lowest opening prices were for the stock in this period.
4. What was the largest change in any one day (based on High and Low price)?
5. What was the largest change between any two days (based on Closing Price)?
6. What was the average daily trading volume during this year?
7. (Optional) What was the median trading volume during this year. (Note: you may need to implement your own function for calculating the median.)

1. Collect data from the Franfurt Stock Exchange, for the ticker AFX_X, for the whole year 2017 (keep in mind that the date format is YYYY-MM-DD).

In [5]:
r_2017 = requests.get('https://www.quandl.com/api/v3/datasets/FSE/AFX_X/data.json?start_date=2017-01-01&end_date=2017-12-31&api_key=' + API_KEY)

2. Convert the returned JSON object into a Python dictionary.

In [6]:
dict_2017 = r_2017.json()

In [7]:
# Extract only the relevant data from the dictionary (data_2017 is a list of lists)
data_2017 = dict_2017['dataset_data']['data']
data_2017[0]

['2017-12-29',
 51.76,
 51.94,
 51.45,
 51.76,
 None,
 34640.0,
 1792304.0,
 None,
 None,
 None]

3. Calculate what the highest and lowest opening prices were for the stock in this period.

In [8]:
# Extract open prices from data_2017
opens = list(zip(*data_2017))[1]

# Calculate the maximum and minimum values in the list of opening prices, ignoring Nonetype values, also record the index
(open_max, i) = max((x, i) for i, x in enumerate(opens) if x is not None)
(open_min, j) = min((y, j) for j, y in enumerate(opens) if y is not None)

# Print the results
print('2017 AFX_X Maximum Open Price: $%.2f on %s' % (open_max, data_2017[i][0]))
print('2017 AFX_X Minimum Open Price: $%.2f on %s' % (open_min, data_2017[j][0]))

2017 AFX_X Maximum Open Price: $53.11 on 2017-12-14
2017 AFX_X Minimum Open Price: $34.00 on 2017-01-24


4. What was the largest change in any one day (based on High and Low price)?

In [9]:
# Extract high and low prices from data_2017
highs = list(zip(*data_2017))[2]
lows = list(zip(*data_2017))[3]

# Calculate the daily variation (difference between the daily high and low)
daily_variation = [x - y for x, y in zip(highs,lows)]

# Find the maximum variation and its index
(max_variation, k) = max((z, k) for k, z in enumerate(daily_variation))

# Print the results
print('2017 AFX_X Maximum Daily Variation: $%.2f on %s' % (max_variation, data_2017[k][0]))

2017 AFX_X Maximum Daily Variation: $2.81 on 2017-05-11


5. What was the largest change between any two days (based on Closing Price)?

In [10]:
# Extract close prices from data_2017
closes = list(zip(*data_2017))[4]

# Calculate the close change between days
daily_close_change = [j-i for i, j in zip(closes[:-1], closes[1:])]

# Calculate the maximum positive and negative daily close change
# Remember the first values in data_2017 are for the last day of the year
(max_pos_daily_close_change, i) = max((x,i) for i, x in enumerate(daily_close_change))
(max_neg_daily_close_change, j) = min((x,j) for j, x in enumerate(daily_close_change))

# Determine if the positive or negative change is greater and print the results
if abs(max_pos_daily_close_change) > abs(max_neg_daily_close_change):
    max_daily_close_change = -max_pos_daily_close_change
    print('2017 AFX_X Maximum Closing Price Day-to-Day Change: -$%.2f on %s to %s' % (abs(max_daily_close_change), data_2017[i+1][0], data_2017[i][0]))
else:
    max_daily_close_change = max_neg_daily_close_change
    print('2017 AFX_X Maximum Closing Price Day-to-Day Change: $%.2f on %s to %s' % (max_daily_close_change, data_2017[j+1][0], data_2017[j][0]))

2017 AFX_X Maximum Closing Price Day-to-Day Change: -$2.56 on 2017-08-08 to 2017-08-09


6. What was the average daily trading volume during this year?

In [11]:
# Extract daily trading volume from data_2017
daily_volume = list(zip(*data_2017))[6]

# Print the calculated mean daily volume
print('2017 AFX_X Average Daily Trading Volume: %.f' % statistics.mean(daily_volume))

2017 AFX_X Average Daily Trading Volume: 89124


7. (Optional) What was the median trading volume during this year. (Note: you may need to implement your own function for calculating the median.)

In [12]:
# Define a function that takes in a list of numbers and calculates the median value
def median(numbers):
    if len(numbers) < 1:
        return None
    elif len(numbers) % 2 == 1:
        return sorted(numbers)[int(len(numbers)/2)]
    else:
        return float(sorted(numbers)[int(len(numbers)/2)] + sorted(numbers)[int(len(numbers)/2-1)])/2

# Call the median function on the daily volume list and print the result
print('2017 AFX_X Median Daily Trading Volume: %.f' % median(daily_volume))

2017 AFX_X Median Daily Trading Volume: 76286
